In [5]:
pip install matplotlib

  Using cached matplotlib-3.4.1-cp37-cp37m-manylinux1_x86_64.whl (10.3 MB)
  Using cached kiwisolver-1.3.1-cp37-cp37m-manylinux1_x86_64.whl (1.1 MB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input
# import tf_slim as slim

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import itertools
import random
%matplotlib inline
np.random.seed(2)

In [7]:
# def SRM(imgs):
#     filter1 = [[0, 0, 0, 0, 0],
#                [0, -1, 2, -1, 0],
#                [0, 2, -4, 2, 0],
#                [0, -1, 2, -1, 0],
#                [0, 0, 0, 0, 0]]
#     filter2 = [[-1, 2, -2, 2, -1],
#                [2, -6, 8, -6, 2],
#                [-2, 8, -12, 8, -2],
#                [2, -6, 8, -6, 2],
#                [-1, 2, -2, 2, -1]]
#     filter3 = [[0, 0, 0, 0, 0],
#                [0, 0, 0, 0, 0],
#                [0, 1, -2, 1, 0],
#                [0, 0, 0, 0, 0],
#                [0, 0, 0, 0, 0]]
#     q = [4.0, 12.0, 2.0]
#     filter1 = np.asarray(filter1, dtype=float) / 4
#     filter2 = np.asarray(filter2, dtype=float) / 12
#     filter3 = np.asarray(filter3, dtype=float) / 2
#     filters = [[filter1, filter1, filter1], [filter2, filter2, filter2], [filter3, filter3, filter3]]
#     filters = np.einsum('klij->ijlk', filters)
#     filters = tf.Variable(filters, dtype=tf.float32)
#     imgs = np.array(imgs, dtype=float)
#     input = tf.Variable(imgs, dtype=tf.float32)
#     op = tf.nn.conv2d(input, filters, strides=[1, 1, 1, 1], padding='SAME')

#     q = [4.0, 12.0, 2.0]
#     filter1 = [[0, 0, 0, 0, 0],
#             [0, -1, 2, -1, 0],
#             [0, 2, -4, 2, 0],
#             [0, -1, 2, -1, 0],
#             [0, 0, 0, 0, 0]]
#     filter2 = [[-1, 2, -2, 2, -1],
#             [2, -6, 8, -6, 2],
#             [-2, 8, -12, 8, -2],
#             [2, -6, 8, -6, 2],
#             [-1, 2, -2, 2, -1]]
#     filter3 = [[0, 0, 0, 0, 0],
#             [0, 0, 0, 0, 0],
#             [0, 1, -2, 1, 0],
#             [0, 0, 0, 0, 0],
#             [0, 0, 0, 0, 0]]
#     filter1 = np.asarray(filter1, dtype=float) / q[0]
#     filter2 = np.asarray(filter2, dtype=float) / q[1]
#     filter3 = np.asarray(filter3, dtype=float) / q[2]
#     filters = [[filter1, filter1, filter1], [filter2, filter2, filter2], [filter3, filter3, filter3]]
#     filters = np.einsum('klij->ijlk', filters)
#     filters = filters.flatten()
#     imgs = np.array(imgs, dtype=float)
#     input = tf.Variable(imgs, dtype=tf.float32)
#     initializer_srm = tf.constant_initializer(filters)
#     def truncate_2(x):
#         neg = ((x + 2) + abs(x + 2)) / 2 - 2
#         return -(-neg+2 + abs(- neg+2)) / 2 + 2
#     op2 = slim.conv2d(input, 3, [5, 5], trainable=False, weights_initializer=initializer_srm, activation_fn=None, padding='SAME', stride=1, scope='srm')
#     op2 = truncate_2(op2)

#     res = np.round(op[0])
#     res[res > 2] = 2
#     res[res < -2] = -2

#     ress = np.array([res], dtype=float)
#     ress2 = np.array(op2, dtype=float)
#     return ress2

In [8]:
def initializer():
    filter1 = [[0, 0, 0, 0, 0],
               [0, -1, 2, -1, 0],
               [0, 2, -4, 2, 0],
               [0, -1, 2, -1, 0],
               [0, 0, 0, 0, 0]]
    filter2 = [[-1, 2, -2, 2, -1],
               [2, -6, 8, -6, 2],
               [-2, 8, -12, 8, -2],
               [2, -6, 8, -6, 2],
               [-1, 2, -2, 2, -1]]
    filter3 = [[0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0],
               [0, 1, -2, 1, 0],
               [0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0]]
    q = [4.0, 12.0, 2.0]
    filter1 = np.asarray(filter1, dtype=float) / 4
    filter2 = np.asarray(filter2, dtype=float) / 12
    filter3 = np.asarray(filter3, dtype=float) / 2
    filters = [[filter1, filter1, filter1], [filter2, filter2, filter2], [filter3, filter3, filter3]]
    filters = np.einsum('klij->ijlk', filters)
    filters = tf.Variable(filters, dtype=tf.float32)
    return filters

In [9]:
h = 224
w = 224
image_size = (h, w)

# np_arr = lambda img: img.resize(image_size).flatten() / 255.0
np_arr = lambda img: np.array(img.resize(image_size)).flatten() / 255.0

def prepare_image(image_path):
    # img = Image.open(image_path)
    # img = img.resize(image_size)
    # img = np.asarray(img)
    # imgs = np.array(img, dtype=float)
    # input = tf.Variable(imgs, dtype=tf.float32)
    return np_arr(Image.open(image_path))
    # SRM([img])

In [10]:
# img = prepare_image('./example_images/Tp_D_CNN_S_O_nat10153_ani00097_12135.jpg')
# img.shape

In [11]:
# img = Image.open('Tp_D_NNN_M_N_pla10121_cha00027_11669.jpg')
# # img = np.asarray(img)
# img = img.resize(image_size)
# img = np.asarray(img) /255.0
# img.shape

In [12]:
# cat = prepare_image('./example_images/Tp_D_CNN_S_O_nat10153_ani00097_12135.jpg')
# plt.imshow(cat[0])
# # plt.show()
# plt.axis('off')
# plt.savefig("srm.png", bbox_inches='tight', pad_inches=0)

In [13]:
X = [] # SRM converted images
Y = [] # 0 for fake, 1 for real

In [14]:
def prepare_data(path, cls):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('jpg') or filename.endswith('png'):
                try:
                    full_path = os.path.join(dirname, filename)
                    X.append(prepare_image(full_path))
                    Y.append(cls)
                except:
                    pass
                if len(Y) % 500 == 0:
                    print('Processing {} images'.format(len(Y)))

In [15]:
#place authentic
Au_path = '../forgery/data/Au'
prepare_data(Au_path, 1)
random.shuffle(X)
# X = X[:2100]
# Y = Y[:2100]
print(len(X), len(Y))

Processing 500 images


/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:766: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag)
/usr/local/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:766: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag)


Processing 1000 images
Processing 1500 images
Processing 2000 images
Processing 2500 images
Processing 3000 images
Processing 3500 images
Processing 4000 images
Processing 4500 images
Processing 5000 images
Processing 5500 images
Processing 6000 images
Processing 6500 images
Processing 7000 images
7329 7329


In [16]:
#place tampered
Tp_path = '../forgery/data/Tp'
prepare_data(Tp_path, 0)
print(len(X), len(Y))

Processing 7500 images
Processing 8000 images
Processing 8500 images
Processing 9000 images
9393 9393


In [17]:
X = np.array(X)
Y = to_categorical(Y, 2)
X = X.reshape(-1, h, w, 3)

x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)

X = X.reshape(-1,1,1,1)

In [18]:
# def truncate_2(x):
#     neg = ((x + 2) + abs(x + 2)) / 2 - 2
#     return -(-neg+2 + abs(- neg+2)) / 2 + 2

In [19]:
initializer_srm = initializer()

In [28]:
input = Input(shape=(h, w, 3))
op = tf.nn.conv2d(input, initializer_srm, strides=[1, 1, 1, 1], padding='SAME', name='srm-layer')
x = Conv2D(32, 3, padding='valid', activation='relu')(op)
x = BatchNormalization()(x)
x = Conv2D(32, 3, padding='valid', activation='relu')(x)
x = MaxPool2D(pool_size=2)(x)
x = Dropout(0.25)(x)
# x = Conv2D(64, 5, padding='valid', activation='relu')(op)
# x = Conv2D(64, 5, padding='valid', activation='relu')(x)
# x = MaxPool2D(pool_size=2)(x)
# x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs=[input], outputs=[output])
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_srm-layer_4 (Ten [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 220, 220, 32)      9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 110, 110, 32)      0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 110, 110, 32)      0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 387200)           

In [29]:
# input = Input(shape=(h,w,3))
# op = tf.nn.conv2d(input, initializer_srm, strides=[1, 1, 1, 1], padding='SAME', name='srm-layer')
# x = Conv2D(32, 3, activation='relu', padding='same')(op)
# x = Conv2D(32, 3, activation='relu', padding='same')(x)
# x = MaxPool2D(padding='same')(x)
# x = Dropout(0.25)(x)

# x = Conv2D(32, 3, activation='relu', padding='same')(x)
# x = Conv2D(32, 3, activation='relu', padding='same')(x)
# x = MaxPool2D(padding='same')(x)
# x = Dropout(0.2)(x)

# x = Conv2D(64, 5, activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = Conv2D(64, 5, activation='relu', padding='same')(x)
# x = MaxPool2D(padding='same')(x)
# x = Dropout(0.25)(x)
# # x = Conv2D(64, 3, activation='relu', padding='same')(x)
# # x = Conv2D(64, 3, activation='relu', padding='same')(x)
# # x = MaxPool2D(padding='same')(x)
# # x = BatchNormalization()(x)

# # x = Conv2D(128, 5, activation='relu', padding='same')(x)
# # x = BatchNormalization()(x)
# # x = Conv2D(128, 5, activation='relu', padding='same')(x)
# # x = MaxPool2D(padding='same')(x)
# # x = Dropout(0.25)(x)

# # x = Conv2D(256, 5, activation='relu', padding='same')(x)
# # x = BatchNormalization()(x)
# # x = Conv2D(256, 5, activation='relu', padding='same')(x)
# # x = MaxPool2D(padding='same')(x)
# # x = Dropout(0.25)(x)

# x = Flatten()(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)

# output = Dense(2, activation='softmax')(x)

# model = tf.keras.Model(inputs=[input], outputs=[output])
# model.summary()

# # model = tf.keras.models.Sequential()
# # model.add(Conv2D(32, 5, padding='valid', activation='relu', input_shape=[224, 224, 3]))
# # model.add(Conv2D(32, 5, padding='valid', activation='relu', input_shape=[224, 224, 3]))
# # model.add(MaxPool2D(pool_size=2))
# # model.add(Dropout(0.25))
# # model.add(Flatten())
# # model.add(Dense(256, activation='relu'))
# # model.add(Dropout(0.5))
# # model.add(Dense(2, activation='softmax'))

# # model.summary()

In [30]:
# input = Input(shape=(h,w,3))
# op = tf.nn.conv2d(input, initializer_srm, strides=[1, 1, 1, 1], padding='SAME', name='srm-layer')
# # op2 = slim.conv2d(input, 3, [5, 5], trainable=False, weights_initializer=initializer_srm, activation_fn=None, padding='SAME', stride=1, scope='srm')
# # op2 = truncate_2(op2)
# x = Conv2D(64, 3, activation='relu', padding='same')(op)
# x = BatchNormalization()(x)
# x = Conv2D(64, 3, activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = MaxPool2D(padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2D(128, 3, activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = Conv2D(128, 3, activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = MaxPool2D(padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2D(256, 5, activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = Conv2D(256, 5, activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = Conv2D(256, 5, activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = MaxPool2D(padding='same')(x)
# x = BatchNormalization()(x)

# x = Conv2D(512, 7, activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = Conv2D(512, 7, activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = Conv2D(512, 7, activation='relu', padding='same')(x)
# x = MaxPool2D(padding='same')(x)
# x = BatchNormalization()(x)

# # x = Conv2D(512, 3, activation='relu', padding='same')(x)
# # x = Conv2D(512, 3, activation='relu', padding='same')(x)
# # x = Conv2D(512, 3, activation='relu', padding='same')(x)

# x = Flatten()(x)
# x = BatchNormalization()(x)
# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)

# output = Dense(2, activation='softmax')(x)

# model = tf.keras.Model(inputs=[input], outputs=[output])
# model.summary()

In [33]:
epochs = 100
batch_size = 32
init_lr = 1e-3
optimizer = Adam(lr = init_lr)
# optimizer = Adam(lr = init_lr, decay = init_lr/epochs)

early_stopping = EarlyStopping(monitor='val_acc', min_delta=0,patience=2, verbose=0, mode='auto')

checkpoint_filepath = 'srm/checkpoint'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True)

model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

In [34]:
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), callbacks=[early_stopping, model_checkpoint_callback])

Epoch 1/100
  2/235 [..............................] - ETA: 10s - loss: 0.0938 - accuracy: 0.9531WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0291s vs `on_train_batch_end` time: 0.0649s). Check your callbacks.


235/235 [==============================] - ETA: 0s - loss: 0.1079 - accuracy: 0.9542WARNING:tensorflow:Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


235/235 [==============================] - 33s 140ms/step - loss: 0.1079 - accuracy: 0.9542 - val_loss: 0.9203 - val_accuracy: 0.8324
Epoch 2/100
235/235 [==============================] - ETA: 0s - loss: 0.0974 - accuracy: 0.9574WARNING:tensorflow:Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


235/235 [==============================] - 32s 138ms/step - loss: 0.0974 - accuracy: 0.9574 - val_loss: 0.8363 - val_accuracy: 0.8334
Epoch 3/100
235/235 [==============================] - ETA: 0s - loss: 0.0899 - accuracy: 0.9649WARNING:tensorflow:Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


235/235 [==============================] - 33s 139ms/step - loss: 0.0899 - accuracy: 0.9649 - val_loss: 0.8826 - val_accuracy: 0.8340
Epoch 4/100
235/235 [==============================] - ETA: 0s - loss: 0.0879 - accuracy: 0.9633WARNING:tensorflow:Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


...

235/235 [==============================] - 20s 86ms/step - loss: 0.0069 - accuracy: 0.9975 - val_loss: 1.6881 - val_accuracy: 0.8547
Epoch 97/100
235/235 [==============================] - ETA: 0s - loss: 0.0101 - accuracy: 0.9968WARNING:tensorflow:Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


235/235 [==============================] - 20s 86ms/step - loss: 0.0101 - accuracy: 0.9968 - val_loss: 1.5945 - val_accuracy: 0.8579
Epoch 98/100
235/235 [==============================] - ETA: 0s - loss: 0.0080 - accuracy: 0.9975WARNING:tensorflow:Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


235/235 [==============================] - 33s 138ms/step - loss: 0.0080 - accuracy: 0.9975 - val_loss: 1.6458 - val_accuracy: 0.8595
Epoch 99/100
235/235 [==============================] - ETA: 0s - loss: 0.0102 - accuracy: 0.9968WARNING:tensorflow:Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


235/235 [==============================] - 20s 86ms/step - loss: 0.0102 - accuracy: 0.9968 - val_loss: 1.5298 - val_accuracy: 0.8590
Epoch 100/100
235/235 [==============================] - ETA: 0s - loss: 0.0083 - accuracy: 0.9969WARNING:tensorflow:Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


235/235 [==============================] - 33s 138ms/step - loss: 0.0083 - accuracy: 0.9969 - val_loss: 1.5227 - val_accuracy: 0.8600


In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")

In [15]:
model.load_weights(checkpoint_filepath)

In [3]:
# def SRM(imgs):
#     c=np.zeros((3,5,5))
#     c[0]=[[-1,2,-2,2,-1],[2,-6,8,-6,2],[-2,8,-12,8,-2],[2,-6,8,-6,2],[-1,2,-2,2,-1]]
#     c[0]=c[0]/12

#     c[1][1][1]=-1
#     c[1][1][2]=2
#     c[1][1][3]=-1
#     c[1][2][1]=2
#     c[1][2][2]=-4
#     c[1][2][3]=2
#     c[1][3][1]=-1
#     c[1][3][2]=2
#     c[1][3][3]=-1
#     c[1]=c[1]/4

#     c[2][1][2]=1
#     c[2][2][2]=-2
#     c[2][3][2]=1
#     c[2]=c[2]/2   

#     Wcnn=np.zeros((5,5,3,3))
#     for i in range(3):
#         Wcnn[:,:,0,i]=c[i]
#         Wcnn[:,:,1,i]=c[i]
#         Wcnn[:,:,2,i]=c[i]

#     imgs = np.array(imgs, dtype=float)
#     input = tf.Variable(imgs, dtype=tf.float32)

#     conv = tf.nn.conv2d(input, Wcnn, [1, 1, 1, 1], padding='SAME',name='srm')
#     res = np.array(conv, dtype=float)
#     return res